#### Test connection

In [28]:
print(client.list_database_names())

['thesis', 'admin', 'local']


#### prep part

In [24]:
!pip3 install pymongo[srv]

In [25]:
from google.colab import drive
drive.mount('/content/drive')
import pymongo
from bson.objectid import ObjectId
import numpy as np

# insert_one(), insert_many(), update_one(), update_many(), replace_one(), delete_one(), delete_many()
# The following links are basic 101 to mongoDB x Python, in Chinese.
# https://blog.csdn.net/culiu9261/article/details/107540063
# https://www.runoob.com/python3/python-mongodb.html

my_access_pwd = "FNhkwCvGlAnRcntG"
my_url = "mongodb+srv://saw008:"+ my_access_pwd + "@cluster0.9vbsx.mongodb.net/sampleTable?retryWrites=true&w=majority"
client = pymongo.MongoClient(my_url)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
# Get all the databases in my cluster.
print(client.list_database_names())
table_data = client['thesis']['data_table']
table_model = client['thesis']['model_table']
table_perf = client['thesis']['performance_table123']

['thesis', 'admin', 'local']


In [27]:
def closest(test_query, num_of_entries = -1):
    # All data and corresponding pattern
    data_list = list(table_data.find({}, {"pattern": 1}))
    if type(test_query) is dict:
        # For stardard form of input dataset
        # the pattern of input data
        test_q_list = list(test_query['pattern'].values())
    elif type(test_query) is list:
        # For normal RAW dataset
        test_q_list = [min(test_query), max(test_query), np.mean(test_query), np.std(test_query)]
    tmp_dict = {}
    for item in data_list:
        tmp_lst = list(item['pattern'].values())
        # Computing Euclidian Dist between test node to each node
        current_dist = ((tmp_lst[0]-test_q_list[0])**2 + \
                        (tmp_lst[1]-test_q_list[1])**2 + \
                        (tmp_lst[2]-test_q_list[2])**2 + \
                        (tmp_lst[3]-test_q_list[3])**2) ** 0.5
                        # (tmp_lst[3]-test_q_list[3])**2 + \
                        # (tmp_lst[4]-test_q_list[4])**2) ** 0.5
        tmp_dict[item['_id']] = current_dist

    # sorted tuple object, which is the ID and corresponding distance.
    L = sorted(tmp_dict.items(), key=lambda item:item[1], reverse=False)

    if num_of_entries > 0:
        L = L[:num_of_entries]

    # turn the list of tuples to a dict
    dictdata = {}
    for l in L:
        dictdata[l[0]] = l[1]    
    return dictdata

def closest_helper(test_query, dataset_list, num_of_entries = -1):
    data_list = []
    for it in dataset_list:
        data_list.append(table_data.find_one({'_id': ObjectId(it)}))
    if type(test_query) is dict:
        # For stardard form of input dataset
        # the pattern of input data
        test_q_list = list(test_query['pattern'].values())
    elif type(test_query) is list:
        # For normal RAW dataset
        test_q_list = [min(test_query), max(test_query), np.mean(test_query), np.std(test_query)]
    tmp_dict = {}
    for item in data_list:
        tmp_lst = list(item['pattern'].values())
        # Computing Euclidian Dist between test node to each node
        current_dist = ((tmp_lst[0]-test_q_list[0])**2 + \
                        (tmp_lst[1]-test_q_list[1])**2 + \
                        (tmp_lst[2]-test_q_list[2])**2 + \
                        (tmp_lst[3]-test_q_list[3])**2) ** 0.5

        tmp_dict[item['_id']] = current_dist

    # sorted tuple object, which is the ID and corresponding distance.
    L = sorted(tmp_dict.items(), key=lambda item:item[1], reverse=False)

    if num_of_entries > 0:
        L = L[:num_of_entries]

    # print(L)
    
    return L[0]


def test_finder(test_query_data, expected_metric, N = -1):
    try:
        # <Step 1>: Get all the similar datasets.
        closest_query_dataID_dist_dict = closest(test_query_data, 10)
        # Top-N near datasets, in form of {dataID1:dist1, dataID2:dist2, ...}


        # <Step 2>: Get all the models who used datasets as training dataset from step1.
        query_id_trainData = list(table_model.find({}, {"training_data":  1}))
        
        # a dict of {modelID: [list_of_training_data]}
        modelID_trainData_dict = {}
        for item in query_id_trainData:
            modelID_trainData_dict[list(item.values())[0]] = list(item.values())[1]
        
        tmp_matched_models = {}
        for trainDataDict_of_one_model in modelID_trainData_dict:
            tmp_set1 = set(closest_query_dataID_dist_dict.keys())
            tmp_set2 = set(modelID_trainData_dict[trainDataDict_of_one_model])
            tmp_intersection = tmp_set1.intersection(tmp_set2)
            if len(tmp_intersection) != 0:
                tmp_matched_models[trainDataDict_of_one_model] = len(tmp_intersection) # matched modelIDs
                # All of the models who have similar-pattern datasets
        # print(len(tmp_matched_models), tmp_matched_models)

        # <Step 3>: Find all the test scenarios / performance who used models in step2.
        matched_perf = []
        tmp_metric = 'task.' + expected_metric
        # print('*****')
        for item in tmp_matched_models.keys():
            matched_perfList_with_one_modelID = list(table_perf.find({'model_id': ObjectId(item)}, {'_id': 1, 'model_id': 1, 'test_data': 1, tmp_metric: 1}))
            if len(matched_perfList_with_one_modelID) == 1:
                # print('single')
                tmp_perf_entry = table_perf.find_one({'model_id': ObjectId(item)}, {'_id': 0, 'model_id': 1, tmp_metric: 1})
                tn = tmp_perf_entry
                if tn is not None:
                    print(tn)
                    matched_perf.append(tn)
            elif len(matched_perfList_with_one_modelID) > 1:
                # print('multiple')
                # For a model has multiple scenarios
                aaa = {}
                for it in matched_perfList_with_one_modelID:
                    # print(it['test_data'])
                    # aaa[it['test_data']] = it['model_id']
                    aaa[it['test_data']] = it['_id']
                    # print(table_data.find_one({'_id': ObjectId('5fbeb895e98c620967f2edac')}))
                # print(matched_perfList_with_one_modelID)
                # print(item)
                closest_testData = closest_helper(test_query_data, aaa.keys())[0]
                # closest_testData_corresponding_modelID = aaa[closest_testData]
                closest_testData_corresponding_perfID = aaa[closest_testData]
                # tn = closest_testData_corresponding_modelID
                tn = closest_testData_corresponding_perfID
                if tn is not None:
                    # print(tn)
                    matched_perf.append(table_perf.find_one({'_id': ObjectId(tn)}, {'_id': 0, 'model_id': 1, tmp_metric: 1}))
            else:
                pass

        # print('*****')
        new_s = sorted(matched_perf, key = lambda e:e.__getitem__('task').__getitem__(expected_metric))
        if N != -1:
            new_s = new_s[:N]

        return new_s # in set/dict format
    except ValueError:
        print('Invalid list, please check again')
        return None

### Run the Algo

In [ ]:
full_data_table = list(table_data.find({}))

tmp_perf_entry = table_perf.find({}, {'test_data': 1, '_id': 0})
tmp_list = list(tmp_perf_entry)
distinct_test_data = set()
for item in tmp_list:
    distinct_test_data.add(item['test_data'])
tmp_distinct = list(distinct_test_data)

# print(len(tmp_distinct))


search = table_data.find_one({'_id': tmp_distinct[1]})
print(search)

# print(search)
print(f"Let's search: {search['pattern']}")
# print(closest(search, -1))
results = test_finder(search, '500-RMSE', 5)
print(len(results), results)

60
{'_id': ObjectId('5fbeb895e98c620967f2ed56'), 'pattern': {'min': 59.9974, 'max': 60.0133, 'avg': 60.0032, 'std': 0.001, 'no_of_samples': 1375}, 'measurement': 'frequency', 'data_path': '/content/drive/My Drive/myThesis/test001/Training/scenario104'}
Let's search: {'min': 59.9974, 'max': 60.0133, 'avg': 60.0032, 'std': 0.001, 'no_of_samples': 1375}
7 {ObjectId('5fc5267ef9ad7c82692b9a88'): 1, ObjectId('5fc526cff9ad7c82692b9a8d'): 1, ObjectId('5fc526cff9ad7c82692b9aa3'): 1, ObjectId('5fc5267ef9ad7c82692b9a81'): 2, ObjectId('5fc526cff9ad7c82692b9a8c'): 1, ObjectId('5fc526cff9ad7c82692b9aa1'): 1, ObjectId('5fc5267ef9ad7c82692b9a89'): 1}
*****
multiple
61afc91cb2235aac34cc2ad2
multiple
61afc962b2235aac34cc2ae9
multiple
61afc962b2235aac34cc2b14
multiple
61afc91cb2235aac34cc2aab
multiple
61afc962b2235aac34cc2ae7
multiple
61afc91cb2235aac34cc2ad8
*****
5 [{'model_id': ObjectId('5fc5267ef9ad7c82692b9a89'), 'task': {'500-RMSE': 0.00054591}}, {'model_id': ObjectId('5fc526cff9ad7c82692b9aa3'), '

### testing

##### Test prep function

In [29]:
def testing(test_query_data, expected_metric, N = -1):
    try:
        # <Step 1>: Get all the similar datasets.
        closest_query_dataID_dist_dict = closest(test_query_data, 10)
        # Top-N near datasets, in form of {dataID1:dist1, dataID2:dist2, ...}


        # <Step 2>: Get all the models who used datasets as training dataset from step1.
        query_id_trainData = list(table_model.find({}, {"training_data":  1}))
        
        # a dict of {modelID: [list_of_training_data]}
        modelID_trainData_dict = {}
        for item in query_id_trainData:
            modelID_trainData_dict[list(item.values())[0]] = list(item.values())[1]
        
        tmp_matched_models = {}
        for trainDataDict_of_one_model in modelID_trainData_dict:
            tmp_set1 = set(closest_query_dataID_dist_dict.keys())
            tmp_set2 = set(modelID_trainData_dict[trainDataDict_of_one_model])
            tmp_intersection = tmp_set1.intersection(tmp_set2)
            if len(tmp_intersection) != 0:
                tmp_matched_models[trainDataDict_of_one_model] = len(tmp_intersection) # matched modelIDs
                # All of the models who have similar-pattern datasets
        # print(len(tmp_matched_models), tmp_matched_models)

        # <Step 3>: Find all the test scenarios / performance who used models in step2.
        matched_perf = []
        tmp_metric = 'task.' + expected_metric
        # print('*****')
        for item in tmp_matched_models.keys():
            matched_perfList_with_one_modelID = list(temp_col.find({'model_id': ObjectId(item)}, {'_id': 1, 'model_id': 1, 'test_data': 1, tmp_metric: 1}))
            if len(matched_perfList_with_one_modelID) == 1:
                # print('single')
                tmp_perf_entry = temp_col.find_one({'model_id': ObjectId(item)}, {'_id': 0, 'model_id': 1, tmp_metric: 1})
                tn = tmp_perf_entry
                if tn is not None:
                    print(tn)
                    matched_perf.append(tn)
            elif len(matched_perfList_with_one_modelID) > 1:
                # print('multiple')
                # For a model has multiple scenarios
                aaa = {}
                for it in matched_perfList_with_one_modelID:
                    # print(it['test_data'])
                    # aaa[it['test_data']] = it['model_id']
                    aaa[it['test_data']] = it['_id']
                    # print(table_data.find_one({'_id': ObjectId('5fbeb895e98c620967f2edac')}))
                # print(matched_perfList_with_one_modelID)
                # print(item)
                closest_testData = closest_helper(test_query_data, aaa.keys())[0]
                # closest_testData_corresponding_modelID = aaa[closest_testData]
                closest_testData_corresponding_perfID = aaa[closest_testData]
                # tn = closest_testData_corresponding_modelID
                tn = closest_testData_corresponding_perfID
                if tn is not None:
                    # print(tn)
                    matched_perf.append(temp_col.find_one({'_id': ObjectId(tn)}, {'_id': 0, 'model_id': 1, tmp_metric: 1}))
            else:
                pass

        # print('*****')
        new_s = sorted(matched_perf, key = lambda e:e.__getitem__('task').__getitem__(expected_metric))
        if N != -1:
            new_s = new_s[:N]

        return new_s # in set/dict format
    except ValueError:
        print('Invalid list, please check again')
        return None

def test_case(dataNo, k, metric):
    client['thesis']['temp'].drop()
    ground_truth = []
    number = dataNo

    tmp_perf_entry = table_perf.find({}, {'test_data': 1, '_id': 0})
    tmp_list = list(tmp_perf_entry)
    distinct_test_data = set()
    for item in tmp_list:
        distinct_test_data.add(item['test_data'])
    tmp_distinct = list(distinct_test_data)
    search = table_data.find_one({'_id': tmp_distinct[number]})

    results = test_finder(search, metric)
    for _ in results:
        ground_truth.append(str(_['model_id']))
    # print(ground_truth)
    # print(len(ground_truth))

    precision_at_k = 0
    recall_at_k = 0
    d = search['_id']
    print('d ->', d)

    my_db = client['thesis']
    my_db.create_collection('temp')
    my_db.list_collection_names()
    mmm = table_perf.find()
    temp_col = my_db['temp']
    temp_col.insert_many(mmm)
    count_before = len(list(temp_col.find()))
    temp_list = list(temp_col.find({'test_data': ObjectId(d)}, {'_id': 1}))
    for item in temp_list:
        temp_col.delete_one({'_id': item['_id']})
    count_after = len(list(temp_col.find()))
    # print(f'Deleted {count_before-count_after} records.')

    # print(f"Let's search: {search['pattern']}")
    # print(closest(search, -1))
    results2 = testing(search, metric, k)
    # print(len(results2), results2)

    found = []
    for _ in results2:
        found.append(str(_['model_id']))
    # print(ground_truth)
    # print(found)
    # print(len(found))
    # print()
    # print(list(results))
    # print(list(results2))
    # print(set(ground_truth).intersection(set(found)))
    precision_at_k = len(set(ground_truth).intersection(set(found))) / k
    recall_at_k = len(set(ground_truth).intersection(set(found))) / len(ground_truth)
    print(f"Precision @ {k} is: {precision_at_k}")
    print(f"Recall @ {k} is: {recall_at_k}")
    print('****************')
    print()

def test_case2(dataNo, k, metric):
    client['thesis']['temp'].drop()
    ground_truth = []
    number = dataNo
    top_what = k

    tmp_perf_entry = table_perf.find({}, {'test_data': 1, '_id': 0})
    tmp_list = list(tmp_perf_entry)
    distinct_test_data = set()
    for item in tmp_list:
        distinct_test_data.add(item['test_data'])
    tmp_distinct = list(distinct_test_data)
    search = table_data.find_one({'_id': tmp_distinct[number]})

    results = test_finder(search, metric, top_what)
    for _ in results:
        ground_truth.append(str(_['model_id']))
    # print(ground_truth)
    # print(len(ground_truth))

    precision_at_k = 0
    recall_at_k = 0
    d = search['_id']
    print('d ->', d)

    my_db = client['thesis']
    my_db.create_collection('temp')
    my_db.list_collection_names()
    mmm = table_perf.find()
    temp_col = my_db['temp']
    temp_col.insert_many(mmm)
    count_before = len(list(temp_col.find()))
    temp_list = list(temp_col.find({'test_data': ObjectId(d)}, {'_id': 1}))
    for item in temp_list:
        temp_col.delete_one({'_id': item['_id']})
    count_after = len(list(temp_col.find()))
    # print(f'Deleted {count_before-count_after} records.')

    print(f"Let's search: {search['pattern']}")
    # print(closest(search, -1))
    results2 = testing(search, metric, top_what)
    # print(len(results2), results2)

    found = []
    for _ in results2:
        found.append(str(_['model_id']))
    abs_gap = []
    for i in range(len(results)):
        ratio = results2[i]['task'][metric] / results[i]['task'][metric]
        abs_gap.append(abs(1-ratio))
    # print(ground_truth)
    # print(found)
    # print(len(found))
    print()
    if len(found) >= 3:
        precision_at_3 = len(set(ground_truth[:3]).intersection(set(found[:3]))) / 3
        recall_at_3 = len(set(ground_truth[:3]).intersection(set(found[:3]))) / len(ground_truth)
        print('---------------------------------')
        print(f"Precision @ 3 is: {round(precision_at_3, 2)}")
        print(f"Recall @ 3 is: {round(recall_at_3, 2)}")
        print(f"Absolute pref gap @ 3 is: {round(np.mean(abs_gap[:3]), 2)}")
    if len(found) >= 5:
        precision_at_5 = len(set(ground_truth[:5]).intersection(set(found[:5]))) / 5
        recall_at_5 = len(set(ground_truth[:5]).intersection(set(found[:5]))) / len(ground_truth)
        print('---------------------------------')
        print(f"Precision @ 5 is: {round(precision_at_5, 2)}")
        print(f"Recall @ 5 is: {round(recall_at_5, 2)}")
        print(f"Absolute pref gap @ 5 is: {round(np.mean(abs_gap[:5]), 2)}")
    if len(found) >= 7:
        precision_at_7 = len(set(ground_truth[:7]).intersection(set(found[:7]))) / 7
        recall_at_7 = len(set(ground_truth[:7]).intersection(set(found[:7]))) / len(ground_truth)
        print('---------------------------------')
        print(f"Precision @ 7 is: {round(precision_at_7, 2)}")
        print(f"Recall @ 7 is: {round(recall_at_7, 2)}")
        print(f"Absolute pref gap @ 7 is: {round(np.mean(abs_gap[:7]), 2)}")

    print('*****************************************************')
    print()
    # print(results)
    # print(results2)

##### test 1

In [30]:
test_case2(19, 3, '500-MAE')
test_case2(3, 5, 'Inference_Time')
test_case2(55, 7, '500-RMSE')
test_case2(48, 9, 'Training_Time')

d -> 5fbeb895e98c620967f2ed75
Let's search: {'min': 59.9977, 'max': 60.0272, 'avg': 60.007, 'std': 0.0034, 'no_of_samples': 1375}

---------------------------------
Precision @ 3 is: 0.67
Recall @ 3 is: 0.67
Absolute pref gap @ 3 is: 0.01
*****************************************************

d -> 5fbeb895e98c620967f2ee1f
Let's search: {'min': 59.9968, 'max': 60.0006, 'avg': 59.9998, 'std': 0.0004, 'no_of_samples': 1375}

---------------------------------
Precision @ 3 is: 1.0
Recall @ 3 is: 0.6
Absolute pref gap @ 3 is: 0.0
---------------------------------
Precision @ 5 is: 1.0
Recall @ 5 is: 1.0
Absolute pref gap @ 5 is: 0.0
*****************************************************

d -> 5fbeb895e98c620967f2ee84
Let's search: {'min': 59.9964, 'max': 60.0153, 'avg': 60.0017, 'std': 0.0017, 'no_of_samples': 1375}

---------------------------------
Precision @ 3 is: 0.67
Recall @ 3 is: 0.29
Absolute pref gap @ 3 is: 0.07
---------------------------------
Precision @ 5 is: 1.0
Recall @ 5 is

###*others*

In [ ]:
from bson.objectid import ObjectId
item = '5fc526cff9ad7c82692b9a8d'
tmp_metric = 'task.' + 'Inference_Time'
tmp_perf_entry = table_perf.find_one({'task.task_category': "prediction"})
# print(tmp_perf_entry)
tmp_perf_entry = table_perf.find({}, {'test_data': 1, '_id': 0})
tmp_list = list(tmp_perf_entry)
print(tmp_list)

distinct_test_data = set()
for item in tmp_list:
    distinct_test_data.add(item['test_data'])
tmp_distinct = list(distinct_test_data)
print(len(tmp_distinct), tmp_distinct)
# print(ObjectId('5fbeb895e98c620967f2ee56') in tmp_distinct)
# print(list(table_perf.find()))

[{'test_data': ObjectId('5fbeb895e98c620967f2ed03')}, {'test_data': ObjectId('5fbeb895e98c620967f2ed11')}, {'test_data': ObjectId('5fbeb895e98c620967f2ed1e')}, {'test_data': ObjectId('5fbeb895e98c620967f2ed9f')}, {'test_data': ObjectId('5fbeb895e98c620967f2edd8')}, {'test_data': ObjectId('5fbeb895e98c620967f2edf5')}, {'test_data': ObjectId('5fbeb895e98c620967f2ee04')}, {'test_data': ObjectId('5fbeb895e98c620967f2ee74')}, {'test_data': ObjectId('5fbeb895e98c620967f2ed1f')}, {'test_data': ObjectId('5fbeb895e98c620967f2ed2d')}, {'test_data': ObjectId('5fbeb895e98c620967f2ede6')}, {'test_data': ObjectId('5fbeb895e98c620967f2ee1f')}, {'test_data': ObjectId('5fbeb895e98c620967f2ee57')}, {'test_data': ObjectId('5fbeb895e98c620967f2ee66')}, {'test_data': ObjectId('5fbeb895e98c620967f2ee84')}, {'test_data': ObjectId('5fbeb895e98c620967f2ee91')}, {'test_data': ObjectId('5fbeb895e98c620967f2ed48')}, {'test_data': ObjectId('5fbeb895e98c620967f2ed91')}, {'test_data': ObjectId('5fbeb895e98c620967f2e

In [15]:
client['thesis']['temp'].drop()
ground_truth = []
number = 9
k = 7
metric = '500-MAE'
top_what = 7

tmp_perf_entry = table_perf.find({}, {'test_data': 1, '_id': 0})
tmp_list = list(tmp_perf_entry)
distinct_test_data = set()
for item in tmp_list:
    distinct_test_data.add(item['test_data'])
tmp_distinct = list(distinct_test_data)
search = table_data.find_one({'_id': tmp_distinct[number]})

results = test_finder(search, metric, top_what)
for _ in results:
    ground_truth.append(str(_['model_id']))
# print(ground_truth)
# print(len(ground_truth))

precision_at_k = 0
recall_at_k = 0
d = search['_id']
print('d ->', d)

my_db = client['thesis']
my_db.create_collection('temp')
my_db.list_collection_names()
mmm = table_perf.find()
temp_col = my_db['temp']
temp_col.insert_many(mmm)
count_before = len(list(temp_col.find()))
temp_list = list(temp_col.find({'test_data': ObjectId(d)}, {'_id': 1}))
for item in temp_list:
    temp_col.delete_one({'_id': item['_id']})
count_after = len(list(temp_col.find()))
# print(f'Deleted {count_before-count_after} records.')

print(f"Let's search: {search['pattern']}")
# print(closest(search, -1))
results2 = testing(search, metric, top_what)
# print(len(results2), results2)

found = []
for _ in results2:
    found.append(str(_['model_id']))
abs_gap = []
for i in range(len(results)):
    ratio = results2[i]['task'][metric] / results[i]['task'][metric]
    abs_gap.append(abs(1-ratio))
# print(ground_truth)
# print(found)
# print(len(found))
print()
if len(found) >= 3:
    precision_at_3 = len(set(ground_truth[:3]).intersection(set(found[:3]))) / 3
    recall_at_3 = len(set(ground_truth[:3]).intersection(set(found[:3]))) / len(ground_truth)
    print('---------------------------------')
    print(f"Precision @ 3 is: {round(precision_at_3, 2)}")
    print(f"Recall @ 3 is: {round(recall_at_3, 2)}")
    print(f"Absolute pref gap @ 3 is: {round(np.mean(abs_gap[:3]), 2)}")
if len(found) >= 5:
    precision_at_5 = len(set(ground_truth[:5]).intersection(set(found[:5]))) / 5
    recall_at_5 = len(set(ground_truth[:5]).intersection(set(found[:5]))) / len(ground_truth)
    print('---------------------------------')
    print(f"Precision @ 5 is: {round(precision_at_5, 2)}")
    print(f"Recall @ 5 is: {round(recall_at_5, 2)}")
    print(f"Absolute pref gap @ 5 is: {round(np.mean(abs_gap[:5]), 2)}")
if len(found) >= 7:
    precision_at_7 = len(set(ground_truth[:7]).intersection(set(found[:7]))) / 7
    recall_at_7 = len(set(ground_truth[:7]).intersection(set(found[:7]))) / len(ground_truth)
    print('---------------------------------')
    print(f"Precision @ 7 is: {round(precision_at_7, 2)}")
    print(f"Recall @ 7 is: {round(recall_at_7, 2)}")
    print(f"Absolute pref gap @ 7 is: {round(np.mean(abs_gap[:7]), 2)}")

print()
# print(results)
# print(results2)

d -> 5fbeb895e98c620967f2ee12
Let's search: {'min': 59.9949, 'max': 60.0018, 'avg': 60.0, 'std': 0.0003, 'no_of_samples': 1375}

---------------------------------
Precision @ 3 is: 0.67
Recall @ 3 is: 0.29
Absolute pref gap @ 3 is: 0.23
---------------------------------
Precision @ 5 is: 1.0
Recall @ 5 is: 0.71
Absolute pref gap @ 5 is: 0.28
---------------------------------
Precision @ 7 is: 1.0
Recall @ 7 is: 1.0
Absolute pref gap @ 7 is: 0.2

